In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob

import pandas as pd

In [3]:
big_data = pd.DataFrame()
files = glob.iglob('backtests/binance/' + '**/backtest_result.txt', recursive=True)
i = 0
for filename in files:
    #print(f'parsing : {filename}')
    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': 'C:\\AgodaGit\\passivbot\\' + filename}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())

        big_data = big_data.append(d, ignore_index=True)

big_data

C:\Users\skhan\AppData\Local\Temp/ipykernel_334964/3439014496.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_334964/3439014496.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_334964/3439014496.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_334964/3439014496.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_

,file,v,s,ebrl,long,short,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrs,tgl_%,tgs_%
0,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.909573,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.114573,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.838780,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.952475,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.242079,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,C:\AgodaGit\passivbot\backtests/binance\XTZUSD...,5.3,XTZUSDT,0.847562,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
491,C:\AgodaGit\passivbot\backtests/binance\XTZUSD...,5.3,XTZUSDT,0.841806,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
492,C:\AgodaGit\passivbot\backtests/binance\XTZUSD...,5.3,XTZUSDT,0.972171,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
493,C:\AgodaGit\passivbot\backtests/binance\XTZUSD...,5.3,XTZUSDT,0.847562,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now filter the dataframe

In [4]:
allowed_symbols = ["ETHUSDT", "XRPUSDT", "1000SHIBUSDT", "DOGEUSDT", "ADAUSDT",
                   "BNBUSDT", "MATICUSDT", "DOTUSDT", "SANDUSDT", "FTMUSDT",
                   "LTCUSDT", "LINKUSDT", "MANAUSDT", "ETCUSDT", "EOSUSDT",
                   "ATOMUSDT", "FILUSDT", "ICPUSDT", "ALICEUSDT", "LRCUSDT",
                   "BCHUSDT", "SUSHIUSDT", "XTZUSDT", "THETAUSDT", "CRVUSDT",
                   "ALGOUSDT", "CHZUSDT", "TRXUSDT", "VETUSDT", "CHRUSDT",
                   "ONEUSDT", "OMGUSDT", "XLMUSDT", "RUNEUSDT", "ENJUSDT",
                   "CELRUSDT", "SXPUSDT", "1INCHUSDT", "SRMUSDT", "COTIUSDT",
                   "ZECUSDT", "QTUMUSDT", "GRTUSDT", "DENTUSDT", "SFPUSDT",
                   "STORJUSDT", "IOTAUSDT", "HOTUSDT", "NEOUSDT", "COMPUSDT",
                   "LINAUSDT", "KAVAUSDT", "BATUSDT", "WAVESUSDT", "IOSTUSDT",
                   "ALPHAUSDT", "RLCUSDT", "HBARUSDT", "DODOUSDT", "XMRUSDT",
                   "SNXUSDT", "ANKRUSDT", "DASHUSDT", "REEFUSDT", "ZENUSDT",
                   "CVCUSDT", "RENUSDT", "ICXUSDT", "RSRUSDT", "SKLUSDT",
                   "NKNUSDT", "BELUSDT", "ONTUSDT", "OGNUSDT", "BLZUSDT",
                   "MTLUSDT", "CTKUSDT", "RVNUSDT", "LITUSDT", "MKRUSDT",
                   "OCEANUSDT", "ZILUSDT", "UNFIUSDT", "BANDUSDT", "YFIIUSDT",
                   "TOMOUSDT", "TRBUSDT", "KNCUSDT", "STMXUSDT", "ZRXUSDT",
                   "SCUSDT", "AKROUSDT", "XEMUSDT", "FLMUSDT", "BALUSDT",
                   "BTSUSDT", "DGBUSDT", "DEFIUSDT"]

#allowed_symbols = ['ETHUSDT']

df = big_data[(big_data.s.isin(allowed_symbols))
              & (big_data['long'] == True)
              & (big_data['short'] == True)

              & (big_data['bnkrl_%'] > 95.0)
              & (big_data['bnkrs_%'] > 95.0)

              & (big_data['v'] == '5.4')

              # & (big_data['tg_%'] > 10)
              # & (big_data['ebr'] > .85)
              ]
df

,file,v,s,ebrl,long,short,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrs,tgl_%,tgs_%
19,C:\AgodaGit\passivbot\backtests/binance\ADAUSD...,5.4,ADAUSDT,1.0000,True,True,0.0000,0.0983,100.0,100.0,0.9940,0.000,0.9025
20,C:\AgodaGit\passivbot\backtests/binance\ADAUSD...,5.4,ADAUSDT,1.0000,True,True,0.0000,0.0983,100.0,100.0,0.9940,0.000,0.9025
21,C:\AgodaGit\passivbot\backtests/binance\ADAUSD...,5.4,ADAUSDT,1.0000,True,True,0.0000,0.0983,100.0,100.0,0.9940,0.000,0.9025
22,C:\AgodaGit\passivbot\backtests/binance\ADAUSD...,5.4,ADAUSDT,1.0000,True,True,0.0000,0.0983,100.0,100.0,0.9940,0.000,0.9025
23,C:\AgodaGit\passivbot\backtests/binance\ADAUSD...,5.4,ADAUSDT,0.9933,True,True,0.0126,0.0487,100.0,100.0,0.9575,0.496,3.3760
24,C:\AgodaGit\passivbot\backtests/binance\ADAUSD...,5.4,ADAUSDT,0.9933,True,True,0.0126,0.0487,100.0,100.0,0.9575,0.496,3.3760
26,C:\AgodaGit\passivbot\backtests/binance\ADAUSD...,5.4,ADAUSDT,0.9763,True,True,0.0271,0.0697,100.0,100.0,0.8892,5.500,14.4900
29,C:\AgodaGit\passivbot\backtests/binance\ADAUSD...,5.4,ADAUSDT,0.9763,True,True,0.0271,0.0697,100.0,100.0,0.8892,5.500,14.4900
31,C:\AgodaGit\passivbot\backtests/binance\ADAUSD...,5.4,ADAUSDT,0.9528,True,True,0.0522,0.0704,100.0,100.0,0.8890,10.820,14.6600
42,C:\AgodaGit\passivbot\backtests/binance\ALGOUS...,5.4,ALGOUSDT,0.9486,True,True,0.0768,0.0381,100.0,100.0,0.9351,17.400,7.6680


In [5]:
df.drop_duplicates(subset=['s', 'tgl_%', 'tgs_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',
                   inplace=True)
df

C:\Users\skhan\AppData\Local\Temp/ipykernel_334964/1022687985.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=['s', 'tgl_%', 'tgs_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',


,file,v,s,ebrl,long,short,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrs,tgl_%,tgs_%
19,C:\AgodaGit\passivbot\backtests/binance\ADAUSD...,5.4,ADAUSDT,1.0000,True,True,0.0000,0.0983,100.0,100.0,0.9940,0.000,0.9025
23,C:\AgodaGit\passivbot\backtests/binance\ADAUSD...,5.4,ADAUSDT,0.9933,True,True,0.0126,0.0487,100.0,100.0,0.9575,0.496,3.3760
26,C:\AgodaGit\passivbot\backtests/binance\ADAUSD...,5.4,ADAUSDT,0.9763,True,True,0.0271,0.0697,100.0,100.0,0.8892,5.500,14.4900
31,C:\AgodaGit\passivbot\backtests/binance\ADAUSD...,5.4,ADAUSDT,0.9528,True,True,0.0522,0.0704,100.0,100.0,0.8890,10.820,14.6600
42,C:\AgodaGit\passivbot\backtests/binance\ALGOUS...,5.4,ALGOUSDT,0.9486,True,True,0.0768,0.0381,100.0,100.0,0.9351,17.400,7.6680
47,C:\AgodaGit\passivbot\backtests/binance\ALGOUS...,5.4,ALGOUSDT,0.9619,True,True,0.0586,0.0515,100.0,100.0,0.9324,12.620,10.3800
64,C:\AgodaGit\passivbot\backtests/binance\ALICEU...,5.4,ALICEUSDT,0.9620,True,True,0.0471,0.0752,100.0,100.0,0.9499,9.747,15.9500
69,C:\AgodaGit\passivbot\backtests/binance\ALICEU...,5.4,ALICEUSDT,0.9400,True,True,0.0596,0.0711,100.0,100.0,0.9392,12.370,14.6400
77,C:\AgodaGit\passivbot\backtests/binance\ATOMUS...,5.4,ATOMUSDT,0.9660,True,True,0.0921,0.0317,100.0,100.0,0.9878,20.420,6.5380
89,C:\AgodaGit\passivbot\backtests/binance\BCHUSD...,5.4,BCHUSDT,0.9745,True,True,0.0219,0.0224,100.0,100.0,0.9886,4.451,4.8610
